<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Проверим на дубликаты.

In [5]:
data.duplicated().sum()

153

Дубликаты есть, необходимо их удалить.

In [6]:
data = data.drop_duplicates()
data = data.reset_index(drop=True)

Переведем данные из типа float в тип integer, так как нецелых чисел в данных нет.

In [7]:
data['Возраст'] = data['Возраст'].astype('int')
data['Зарплата'] = data['Зарплата'].astype('int')

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                4847 non-null   int64
 1   Возраст            4847 non-null   int64
 2   Зарплата           4847 non-null   int64
 3   Члены семьи        4847 non-null   int64
 4   Страховые выплаты  4847 non-null   int64
dtypes: int64(5)
memory usage: 189.5 KB


Мы изучили даные и подготовили их к работе.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос**: Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Нет.

**Обоснование:** Докажем, что при домножении на обратимую матрицу P предсказания не изменятся.
При доказательстве мы будем использовать следующие свойства:

* $(AB)^T = B^T A^T$

* $(AB)^{-1} = B^{-1} A^{-1}$

* $AA^{-1} = A^{-1}A = E$

Домножаем X на P

$$ a_{new} = XPw $$

$$ w = ((XP)^T XP)^{-1} XP^T y $$ 

По свойствам раскрываем скобки

$$ a_{new} = XP ((XP)^T XP)^{-1} (XP)^T y = XP (P^T X^T XP)^{-1} P^T X^T y = XPP^{-1} (P^T X^T X)^{-1} P^T X^T y$$
$$ a_{new} = XPP^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y = X (X^T X)^{-1} X^T y = Xw $$

Видим, что при доможении предсказания не изменились.

Но в случае, если матрица P необратима, например, она неквадратная или вырожденная, такие преобразования мы произвести не можем, соответственно на такую матрицу домножать нельзя.


## Алгоритм преобразования

**Алгоритм**
* Создаем случайную матрицу Р размера nхn, где n - это количество признаков.
* Проверяем ее на обратимость - вычисляем детерминант. Если он не равен 0 - матрица обратима.
* Перемножаем X и Р и далее работаем с результатом - зашифрованной матрицей М.
* Для восстановления матрицы X нужно зашифрованную матрицу М умножить на обратную к P.

**Обоснование**

Создадим матрицы X(3х2) и Р(2х2). Размер матрицы Р не может быть никаким другим, так как иначе мы не сможем перемножать матрицы.

In [9]:
X = np.array([[1, 2], [3, 4], [5, 6]])
P = np.array([[1, 2], [3, 4]])

Вычисляем детерминант, чтобы удостовериться, что матрица обратима.

In [10]:
np.linalg.det(P)

-2.0000000000000004

Детерминант не равен нулю, матрица обратима, можно шифровать нашу матрицу.

In [11]:
M = X @ P
M

array([[ 7, 10],
       [15, 22],
       [23, 34]])

Для обратной расшифровки данных домножаем матрицу М на обратную Р.

In [12]:
P_inv = np.linalg.inv(P)
M @ P_inv

array([[1., 2.],
       [3., 4.],
       [5., 6.]])

Получаем исходную матрицу.

Соберем алгоритм шифрования в виде класса.

In [13]:
class Crypto:
    """
    Шифрует и дешифрует матрицы
    """
            
    def cifer(self, X):
        self.det = 0
        n = X.shape[1]
        while self.det == 0:
            self.P = np.random.randint(1, 10, (n,n))
            self.det = np.linalg.det(self.P)
        self.P_inv = np.linalg.inv(self.P)
        
        return X @ self.P
    
    def decifer(self, M):
        return M @ self.P_inv

## Проверка алгоритма

Выделим из данных целевой признак.

In [14]:
y = data['Страховые выплаты']
X = data.drop('Страховые выплаты', axis=1)

Обучим линейную регрессию на незашифрованных данных и оценим качество.

In [15]:
linreg_X = LinearRegression()
linreg_X.fit(X, y)
pred_X = linreg_X.predict(X)
r2_X = r2_score(y, pred_X)
print('R2 для данных без шифрования', r2_X)

R2 для данных без шифрования 0.4302010046633359


Теперь зашифруем данные и снова обучим линейную регрессию и оценим качество.

In [16]:
crypto = Crypto()
M = crypto.cifer(X)

In [17]:
linreg_M = LinearRegression()
linreg_M.fit(M, y)
pred_M = linreg_M.predict(M)
r2_M = r2_score(y, pred_M)
print('R2 для зашифрованных данных',r2_M)

R2 для зашифрованных данных 0.430201004663338


Сравниваем качество.

In [18]:
np.isclose(r2_X, r2_M)

True

Качество не изменилось. Можно использовать наш алгоритм для шифрования данных.